In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.linear_model import LinearRegression

## Load the data

In [17]:
data = pd.read_csv('1.02. Multiple linear regression.csv')
data.head()

,SAT,"Rand 1,2,3",GPA
0,1714,1,2.40
1,1664,3,2.52
2,1760,3,2.54
3,1685,3,2.74
4,1693,2,2.83


In [18]:
data.describe()

,SAT,"Rand 1,2,3",GPA
count,84.000000,84.000000,84.000000
mean,1845.273810,2.059524,3.330238
std,104.530661,0.855192,0.271617
min,1634.000000,1.000000,2.400000
25%,1772.000000,1.000000,3.190000
50%,1846.000000,2.000000,3.380000
75%,1934.000000,3.000000,3.502500
max,2050.000000,3.000000,3.810000


## Create the multiple linear regression

In [19]:
x = data[['SAT','Rand 1,2,3']]
y = data['GPA']

### Working out the regression formula

In [20]:
reg = LinearRegression()
reg.fit(x,y)

LinearRegression()

In [21]:
reg.coef_

array([ 0.00165354, -0.00826982])

In [22]:
reg.intercept_

0.29603261264909486

## Calculating the R-squared

In [23]:
reg.score(x,y)

0.40668119528142843

### Formula for Adjusted R^2
$R^2_{adj.} = 1 - (1-R^2)*\frac{n-1}{n-p-1}$

In [24]:
x.shape[0]

84

In [25]:
r2 = reg.score(x,y)
rAdjusted = 1 - (1 - r2)*(x.shape[0] - 1)/(x.shape[0] - x.shape[1]-1)
rAdjusted

0.39203134825134023

## Feature selection

In [26]:
from sklearn.feature_selection import f_regression

In [27]:
f_regression(x,y)

(array([56.04804786,  0.17558437]), array([7.19951844e-11, 6.76291372e-01]))

In [28]:
p_values = f_regression(x,y)[1]
p_values

array([7.19951844e-11, 6.76291372e-01])

In [29]:
p_values.round(3)

array([0.   , 0.676])

## Creating a summary table

In [30]:
reg_summary = pd.DataFrame(data = x.columns.values, columns=['Features'])
reg_summary

,Features
0,SAT
1,"Rand 1,2,3"


In [31]:
reg_summary['Coefficients'] = reg.coef_
reg_summary['p-values'] = p_values.round(3)
reg_summary

,Features,Coefficients,p-values
0,SAT,0.001654,0.000
1,"Rand 1,2,3",-0.008270,0.676


## Standardization


In [32]:
from sklearn.preprocessing import StandardScaler

In [35]:
scaler = StandardScaler()
scaler

StandardScaler()

In [36]:
scaler.fit(x)

StandardScaler()

In [47]:
x_scaled = scaler.transform(x)

In [66]:
x_scaled

array([[-1.26338288, -1.24637147],
       [-1.74458431,  1.10632974],
       [-0.82067757,  1.10632974],
       [-1.54247971,  1.10632974],
       [-1.46548748, -0.07002087],
       [-1.68684014, -1.24637147],
       [-0.78218146, -0.07002087],
       [-0.78218146, -1.24637147],
       [-0.51270866, -0.07002087],
       [ 0.04548499,  1.10632974],
       [-1.06127829,  1.10632974],
       [-0.67631715, -0.07002087],
       [-1.06127829, -1.24637147],
       [-1.28263094,  1.10632974],
       [-0.6955652 , -0.07002087],
       [ 0.25721362, -0.07002087],
       [-0.86879772,  1.10632974],
       [-1.64834403, -0.07002087],
       [-0.03150724,  1.10632974],
       [-0.57045283,  1.10632974],
       [-0.81105355,  1.10632974],
       [-1.18639066,  1.10632974],
       [-1.75420834,  1.10632974],
       [-1.52323165, -1.24637147],
       [ 1.23886453, -1.24637147],
       [-0.18549169, -1.24637147],
       [-0.5608288 , -1.24637147],
       [-0.23361183,  1.10632974],
       [ 1.68156984,

## Regression with scaled features

In [49]:
reg = LinearRegression()
reg.fit(x_scaled,y)

LinearRegression()

In [50]:
reg.coef_

array([ 0.17181389, -0.00703007])

In [51]:
reg.intercept_

3.330238095238095

## Creating a summary table

In [59]:
reg_summary = pd.DataFrame(data=[['Bias'],['SAT'],['Rand 1,2,3']], columns=['Features'])
reg_summary['Weights'] = reg.intercept_, reg.coef_[0], reg.coef_[1]

In [60]:
reg_summary

,Features,Weights
0,Bias,3.330238
1,SAT,0.171814
2,"Rand 1,2,3",-0.007030


## Making predictions with the standardized coefficients

In [62]:
new_data = pd.DataFrame(data=[[1700,2],[1452,1]], columns=['SAT','Rand 1,2,3'])
new_data

,SAT,"Rand 1,2,3"
0,1700,2
1,1452,1


In [63]:
reg.predict(new_data)

array([295.39979563, 252.79698029])

In [64]:
new_data_scaled = scaler.transform(new_data)
new_data_scaled

array([[-1.39811928, -0.07002087],
       [-3.78487835, -1.24637147]])

In [65]:
reg.predict(new_data_scaled)

array([3.09051403, 2.68870549])

## What if we removed the 'Rand 1,2,3' variable?

In [68]:
reg_simple = LinearRegression()
x_simple_matrix = x_scaled[:,0].reshape(-1,1)
reg_simple.fit(x_simple_matrix,y)

LinearRegression()

In [69]:
reg_simple.predict(new_data_scaled[:,0].reshape(-1,1))

array([3.08970998, 2.67909935])